In [2]:
import warnings
warnings.filterwarnings('ignore')

In [112]:
import requests
import urllib
from bs4 import BeautifulSoup
import random
import time
from tqdm import tqdm_notebook as tqdm
import re
import unicodedata
import collections
import re
import string
import json
import pandas as pd
import numpy as np
from nltk import ngrams

import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import seaborn as sns

from pymystem3 import Mystem
mystem = Mystem()

SEED=1337

import matplotlib.pylab as plt
%matplotlib inline

from sklearn.cross_validation import train_test_split as train
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [89]:
df = pd.read_csv('/Users/alinashaymardanova/Desktop/Диплом/data/result.csv', sep='\t', encoding='utf-8')

In [90]:
del df['Unnamed: 0']
del df['Frequency word']
del df['Poem']
del df['Len']
del df['Stanzas_cos']
del df['Stanzas_eucl']
del df['Stanzas_dot']

In [93]:
X = df.drop(('Author'), axis=1)
y = df['Author']
feature_names = X.columns

In [95]:
X_train, X_test, y_train, y_test = train(X, y, test_size=0.6, random_state = 11)

In [96]:
rf = RandomForestClassifier(
    random_state=SEED,
    max_depth=100,
    n_estimators=500,
    class_weight='balanced'
)

In [97]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=100, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=500, n_jobs=None, oob_score=False,
                       random_state=1337, verbose=0, warm_start=False)

In [108]:
text = 'рассказ о голубом просторе\nпоющее ядро зенит\nмондео задний бампер в сборе\nразмер аэродромных плит\nкак заработать в интернете\nесть ли акулы на пхукете\nтекст в доме пусто я один\nвсе о роддоме для мужчин\nгде в ломоносовском районе\nизосорбидадинитрат \nземлеустройство реферат \nкартошка на мясном бульоне\nиз мавритании жених \nтатуировки на двоих'

In [4]:
from pymystem3 import Mystem

import requests
import re

url = 'https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map'

mapping = {}
r = requests.get(url, stream=True)
for pair in r.text.split('\n'):
    pair = re.sub('\s+', ' ', pair, flags=re.U).split(' ')
    if len(pair) > 1:
        mapping[pair[0]] = pair[1]
        
def tag_mystem(text):  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            if pos in mapping:
                tagged.append(lemma + '_' + mapping[pos]) # здесь мы конвертируем тэги
            else:
                tagged.append(lemma + '_X') # на случай, если попадется тэг, которого нет в маппинге
        except KeyError:
            continue # я здесь пропускаю знаки препинания, но вы можете поступить по-другому

    return tagged

import numpy as np
from scipy import spatial

model = gensim.models.KeyedVectors.load_word2vec_format('/Users/alinashaymardanova/Downloads/180/model.bin', binary=True)


from gensim import matutils

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)


from scipy.spatial import distance
from scipy.spatial.distance import cosine


def get_sim_vect(tag_teat):
    v_sim = list()
    v_cos = list()
    v_eucl = list()
    if len(tag_teat) > 1:
        for word in tag_teat:
            i = -1
            while i <= len(tag_teat):
                i += 1
                try:
                    v_sim.append(similarity(model[word], model[tag_teat[i]]))
                    v_eucl.append(distance.euclidean(model[word], model[tag_teat[i]]))
                    v_cos.append(cosine(model[word], model[tag_teat[i]]))
                except:
                    continue
    
    return  v_sim, v_cos, v_eucl


def get_ADV(tag_teat):
    adv = [word for word in tag_teat if re.search("ADV|ADJ", word)]
    return adv

  

def get_sanz_vectors(vectors):
    v_sim = list()
    v_cos = list()
    v_eucl = list()
    if len(vectors) != 1:
        for word in vectors:
            i = -1
            while i <= len(vectors):
                i += 1
                try:
                    v_sim.append(similarity(word, vectors[i]))
                    v_eucl.append(distance.euclidean(word, vectors[i]))
                    v_cos.append(cosine(word, vectors[i]))
                except:
                    continue

    return v_sim, v_cos,v_eucl


def get_ngram_without_spase(text, N):
    
    ngram = list()
    for word in text.split():
        ngr = ngrams(word, N)
        
        for el in ngr:
            ngram.append(el)
    
    e = collections.Counter(ngram)
    
    return (e[sorted(e, key=e.get, reverse=True)[0]])/len(e)


from itertools import islice, tee

def get_ngram(content_norm, N):
    three_grams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(content_norm, N))))
    three_grams = [''.join(x) for x in three_grams]
     
    e = collections.Counter(three_grams)
    return (e[sorted(e, key=e.get, reverse=True)[0]])/len(e)

In [113]:
def analysis(text):
    text = input()
    df = pd.DataFrame()
    try:
        clean_text = normalize(text)
        tag_text = tag_mystem(''.join(clean_text))
        v_sim, v_cos, v_eucl = get_sim_vect(tag_text)


        adv = get_ADV(tag_text)
        v_sim_adv, v_cos_adv, v_eucl_adv = get_sim_vect(adv)


        three = get_ngram(clean_text, 2)
        two = get_ngram(clean_text, 3)

        two_gram = get_ngram_without_spase(clean_text, 2)
        three_gram = get_ngram_without_spase(clean_text, 3)
        two_gram_s = get_ngram(clean_text, 2)
        three_gram_s = get_ngram(clean_text, 3)


        df =  df.append({'Semantic_density_dot': np.mean(v_sim),
                         'Semantic_density_cos': np.mean(v_cos),
                         'Semantic_density_eucl': np.mean(v_eucl),
                         'Adj_semantic_density_dot': np.mean(v_sim_adv),
                         'Adj_semantic_density_cos': np.mean(v_cos_adv),
                         'Adj_semantic_density_eucl': np.mean(v_eucl_adv),
                         'Bigramms_without_spaces': two_gram,
                         'ThreeGrams_without_spaces': three_gram,
                        'Bigramms_with_spaces': two_gram_s,
                        'ThreeGrams_with_spaces': three_gram_s}, 
                        ignore_index=True)
    except:
        print('Упс, что-то пошло не так:(')
        
    return rf.predict(df)[0]